## Evaluation of Adnet Client Requests

### Statistics of Interest
#### Setting
- we have tested for
  - 40 potential vulnerabilities/configurations
  - 7 different algorithms (required also for vulnerability assessments)
  - (explicit session finish)
  - ~legacy downgrade vulnerabilities~ (contained in the 40 vulnerabilities)

#### Statistics and Plots
- "impact" of the different vulnerabilities
  - as expressed by share of users vulnerable to them
  - represented by ???-plot (40 vulns to cover...)
- "competibility" of the different vulnerabilities
  - as expressed by their share among the total number of individual vulnerabilities found
- 

### Evaluations Sketch
- load request log file to DF
- create complete request matrix for each of the resolvers and domains (ip, d1, d2, d3, d4)
- determine from request matrix for each (resolver, user)-tuple
  - has session finised
  - for each algorithm: supports algorithm
  - validates dnssec ("broken"; combination of algorithm support)
  - for each vulnerability check: is vulnerable

In [1]:
import pandas as pd
import numpy as np
from urllib import parse
# import logging
import matplotlib as plt
import seaborn as sns
import matplotlib.ticker as mtick

# pd.options.display.max_rows = 2000
# pd.options.display.max_columns = 2000
# logging.getLogger('matplotlib.font_manager').disabled = True

pd.options.plotting.backend = 'matplotlib'
plt.style.use('ggplot')

REPO_DIR = '../../dnssec-downgrade-data/'
DATA_DIR = REPO_DIR + '/2021-10-06_adnet-study/'  # location of input/raw and processed data
STATS_DIR = DATA_DIR + '/stats/' # output location fo tables and plots 

IP_VICTIM = '104.238.214.165'
IP_ATTACKER = '104.238.214.154'

LOGFILE_DEV = DATA_DIR + '/dev-adnet.json'  # proper subset of EU logfile
LOGFILE_EU = DATA_DIR + '/downg-EU.json'
LOGFILE_AF = DATA_DIR + '/downg-AF.json'
LOGFILE_SA = DATA_DIR + '/downg-SA.json'
LOGFILE_NA = DATA_DIR + '/downg-NA.json'
LOGFILE_OC = DATA_DIR + '/downg-OC.json'
LOGFILE_AS = DATA_DIR + '/downg-AS.json'

# Regions
R_DEV = "dev"
R_EU = "eu"  # Europe
R_AF = "af"  # Africa
R_SA = "sa"  # South America
R_NA = "na"  # North America
R_OC = "oc"  # Oceania
R_AS = "as"  # Asia
REGIONS = [
    # R_DEV,
    R_EU,
    R_AF,
    # R_SA,
    # R_NA,
    # R_OC,
    # R_AS,
]
REGIONS_TO_LOGFILES = {
    R_DEV: LOGFILE_DEV,
    R_EU: LOGFILE_EU,
    R_AF: LOGFILE_AF,
    R_SA: LOGFILE_SA,
    R_NA: LOGFILE_NA,
    R_OC: LOGFILE_OC,
    R_AS: LOGFILE_AS,
}


STUDY_DOMAINS = {  # Must not be FQDN!
    "resolver-downgrade-attack.dedyn.io",
    "downgrade.dedyn.io"
}

TEST_NAMES = {
    # Downgrade Vulnerabilities
    "mitm-ra-ds8-ds13.ds13-ds16-dnskey13-dnskey16",
    "mitm-ra-ds8-ds13.ds8-ds15-dnskey15",
    "mitm-ra-ds8-ds13-ds15.ds13-ds16-dnskey13-dnskey16",
    "mitm-ra-ds8-ds13-ds15.ds16",
    "mitm-ra-ds8-ds13-ds15.ds8-ds15-dnskey15",
    "mitm-ra-ds8-ds13-ds15-ds16.ds8-ds16-dnskey16",
    "mitm-ra-ds8-ds13-ds15-ds16.ds16",
    "mitm-ra-ds8-ds13-ds15-ds16.ds8-ds15-dnskey15",
    "mitm-rs13-ra.ds8-ds16-dnskey16",
    "mitm-rs13-ra.ds8-ds13-dnskey8",
    "mitm-rs13-ra.ds8-ds15-dnskey8-dnskey15",
    "mitm-rs13-ra.ds13-ds16-dnskey13-dnskey16",
    "mitm-rs13-ra.ds16",
    "mitm-rs15-ra.ds8-ds16-dnskey16",
    "mitm-rs15-ra.ds8-ds13-dnskey13",
    "mitm-rs15-ra.ds8-dnskey8",
    "mitm-rs15-ra.ds8-ds16-dnskey8-dnskey16",
    "mitm-rs15-ra.ds13-ds16-dnskey13-dnskey16",
    "mitm-rs15-ra.ds16",
    "mitm-rs15-ra.ds8-ds16-dnskey16",
    "mitm-rs15-ra.ds8-ds15-dnskey15",
    "mitm-rs16-ra.ds8-ds13-dnskey13",
    "mitm-rs16-ra.ds15-ds16-dnskey15",
    "mitm-rs16-ra.ds13-ds16-dnskey16",
    "mitm-rs16-ra.ds13-ds16-dnskey13-dnskey16",
    "mitm-rs16-ra.ds8-ds13-dnskey8",
    "mitm-rs16-ra.ds8-ds15-dnskey8-dnskey15",
    "mitm-rs16-ra.ds13-dnskey13",
    "mitm-rs16-ra.ds8-ds13-dnskey8-dnskey13",
    "mitm-rs16-ra.ds13-ds15-dnskey15",
    "mitm-rs8-ra.ds8-ds16-dnskey16",
    "mitm-rs8-ra.ds15-ds16-dnskey16",
    "mitm-rs8-ra.ds13-ds16-dnskey16",
    "mitm-rs8-ra.ds13-ds16-dnskey13-dnskey16",
    "mitm-rs8-ra.ds13-dnskey13",
    "mitm-rs8-ra.ds13-ds15-dnskey13-dnskey15",
    "mitm-rs8-ra.ds8-ds13-dnskey8-dnskey13",
    "mitm-rs8-ra.ds8-dnskey8",
    "mitm-rs8-ra.ds16",
    "mitm-rs8-ra.ds8-ds16-dnskey8-dnskey16",
    
    # Algorithm Support
    "mitm-ra.ds5-dnskey5",
    "mitm-ra.ds8-dnskey8",
    "mitm-ra.ds10-dnskey10",
    "mitm-ra.ds13-dnskey13",
    "mitm-ra.ds14-dnskey14",
    "mitm-ra.ds15-dnskey15",
    "mitm-ra.ds16-dnskey16",
    
    # Legacy Downgrade Vulnerabilities (actually covered by non-legacy)
    # "ecdsap256sha256",
    # "onlyrsasha256",
    # "rsasha256",
    
    # Housekeeping
    "broken",
    "session-finish",  # substitute for empty child / parent domain
}

### Load Request Data

In [2]:
region_dfs = [pd.read_json(REGIONS_TO_LOGFILES[region], lines=True) for region in REGIONS]

df_req_raw = pd.concat(region_dfs, keys=REGIONS).reset_index(level=0).rename(mapper={"level_0": "region"}, axis=1)
df_req_raw

,region,time_epoch,time_human,ip_server,ip_client,request_method,protocol,host_header,server_alias,port_server,url_path,filename,query,time_served_ms,status,errlog_reqest_id,user_agent
0,eu,1633511250653,2021-10-06T09:07:30,104.238.214.154,194.230.144.141,GET,HTTP/1.1,mitm-rs8-ra.ds13-dnskey13.downgrade.dedyn.io,ds13-dnskey13.downgrade.dedyn.io,443,/img.png,/var/www/downgrade.dedyn.io/img.png,?test=mitm-rs8-ra.ds13-dnskey13&tok=730252720&...,271,200,-,Mozilla/5.0 (iPhone; CPU iPhone OS 14_7_1 like...
1,eu,1633511250746,2021-10-06T09:07:30,104.238.214.154,194.230.144.141,GET,HTTP/1.1,mitm-rs8-ra.ds13-ds16-dnskey13-dnskey16.downgr...,ds13-ds16-dnskey13-dnskey16.downgrade.dedyn.io,443,/img.png,/var/www/downgrade.dedyn.io/img.png,?test=mitm-rs8-ra.ds13-ds16-dnskey13-dnskey16&...,291,200,-,Mozilla/5.0 (iPhone; CPU iPhone OS 14_7_1 like...
2,eu,1633511250854,2021-10-06T09:07:30,104.238.214.154,194.230.144.141,GET,HTTP/1.1,mitm-ra-ds8-ds13-ds15.ds16.downgrade.dedyn.io,ds16.downgrade.dedyn.io,443,/img.png,/var/www/downgrade.dedyn.io/img.png,?test=mitm-ra-ds8-ds13-ds15.ds16&tok=730252720...,466,200,-,Mozilla/5.0 (iPhone; CPU iPhone OS 14_7_1 like...
3,eu,1633511257590,2021-10-06T09:07:37,104.238.214.154,188.216.95.240,GET,HTTP/1.1,mitm-ra-ds8-ds13.ds13-ds16-dnskey13-dnskey16.d...,ds13-ds16-dnskey13-dnskey16.downgrade.dedyn.io,443,/img.png,/var/www/downgrade.dedyn.io/img.png,?test=mitm-ra-ds8-ds13.ds13-ds16-dnskey13-dnsk...,406,200,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
4,eu,1633511257721,2021-10-06T09:07:37,104.238.214.154,188.216.95.240,GET,HTTP/1.1,mitm-ra-ds8-ds13.ds8-ds15-dnskey15.downgrade.d...,ds8-ds15-dnskey15.downgrade.dedyn.io,443,/img.png,/var/www/downgrade.dedyn.io/img.png,?test=mitm-ra-ds8-ds13.ds8-ds15-dnskey15&tok=2...,401,200,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23893,af,1633529284304,2021-10-06T14:08:04,104.238.214.154,197.40.127.184,GET,HTTP/1.1,mitm-ra-ds8-ds13.ds13-ds16-dnskey13-dnskey16.d...,ds13-ds16-dnskey13-dnskey16.downgrade.dedyn.io,443,/img.png,/var/www/downgrade.dedyn.io/img.png,?test=mitm-ra-ds8-ds13.ds13-ds16-dnskey13-dnsk...,417,200,-,Mozilla/5.0 (Linux; Android 8.1.0; SM-T585) Ap...
23894,af,1633529284328,2021-10-06T14:08:04,104.238.214.154,197.40.127.184,GET,HTTP/1.1,mitm-ra-ds8-ds13.ds8-ds15-dnskey15.downgrade.d...,ds8-ds15-dnskey15.downgrade.dedyn.io,443,/img.png,/var/www/downgrade.dedyn.io/img.png,?test=mitm-ra-ds8-ds13.ds8-ds15-dnskey15&tok=2...,459,200,-,Mozilla/5.0 (Linux; Android 8.1.0; SM-T585) Ap...
23895,af,1633529285064,2021-10-06T14:08:05,104.238.214.154,197.40.127.184,GET,HTTP/1.1,mitm-ra-ds8-ds13-ds15.ds16.downgrade.dedyn.io,ds16.downgrade.dedyn.io,443,/img.png,/var/www/downgrade.dedyn.io/img.png,?test=mitm-ra-ds8-ds13-ds15.ds16&tok=239304664...,333,200,-,Mozilla/5.0 (Linux; Android 8.1.0; SM-T585) Ap...
23896,af,1633529285237,2021-10-06T14:08:05,104.238.214.154,197.40.127.184,GET,HTTP/1.1,mitm-ra-ds8-ds13-ds15.ds8-ds15-dnskey15.downgr...,ds8-ds15-dnskey15.downgrade.dedyn.io,443,/img.png,/var/www/downgrade.dedyn.io/img.png,?test=mitm-ra-ds8-ds13-ds15.ds8-ds15-dnskey15&...,353,200,-,Mozilla/5.0 (Linux; Android 8.1.0; SM-T585) Ap...


#### Remove Irrelevant Columns

In [3]:
df_req_reduced = df_req_raw[['region', 'query', 'ip_server']]
df_req_reduced

,region,query,ip_server
0,eu,?test=mitm-rs8-ra.ds13-dnskey13&tok=730252720&...,104.238.214.154
1,eu,?test=mitm-rs8-ra.ds13-ds16-dnskey13-dnskey16&...,104.238.214.154
2,eu,?test=mitm-ra-ds8-ds13-ds15.ds16&tok=730252720...,104.238.214.154
3,eu,?test=mitm-ra-ds8-ds13.ds13-ds16-dnskey13-dnsk...,104.238.214.154
4,eu,?test=mitm-ra-ds8-ds13.ds8-ds15-dnskey15&tok=2...,104.238.214.154
...,...,...,...
23893,af,?test=mitm-ra-ds8-ds13.ds13-ds16-dnskey13-dnsk...,104.238.214.154
23894,af,?test=mitm-ra-ds8-ds13.ds8-ds15-dnskey15&tok=2...,104.238.214.154
23895,af,?test=mitm-ra-ds8-ds13-ds15.ds16&tok=239304664...,104.238.214.154
23896,af,?test=mitm-ra-ds8-ds13-ds15.ds8-ds15-dnskey15&...,104.238.214.154


#### Expand Data From Query Parameter

In [4]:
def splitq(df):
    test = np.nan
    token = np.nan
    time = np.nan
    query = df['query']
    if query is not None and len(query) > 1:
        params = dict(parse.parse_qsl(query[1:]))
        test = params.get('test')
        token = str(params.get('tok'))
        time = params.get('time')
    return test, token, time
        

df_req_qsplit = df_req_reduced.copy(deep=True)
df_req_qsplit[['test', 'token', 'time_client']] = df_req_qsplit.apply(axis=1, func=splitq, result_type='expand')
df_req_qsplit = df_req_qsplit.drop(columns=['query'])
df_req_qsplit

,region,ip_server,test,token,time_client
0,eu,104.238.214.154,mitm-rs8-ra.ds13-dnskey13,730252720,1633511249822
1,eu,104.238.214.154,mitm-rs8-ra.ds13-ds16-dnskey13-dnskey16,730252720,1633511249822
2,eu,104.238.214.154,mitm-ra-ds8-ds13-ds15.ds16,730252720,1633511249820
3,eu,104.238.214.154,mitm-ra-ds8-ds13.ds13-ds16-dnskey13-dnskey16,2977949033,1633511256649
4,eu,104.238.214.154,mitm-ra-ds8-ds13.ds8-ds15-dnskey15,2977949033,1633511256649
...,...,...,...,...,...
23893,af,104.238.214.154,mitm-ra-ds8-ds13.ds13-ds16-dnskey13-dnskey16,2393046648,1633529278319
23894,af,104.238.214.154,mitm-ra-ds8-ds13.ds8-ds15-dnskey15,2393046648,1633529278321
23895,af,104.238.214.154,mitm-ra-ds8-ds13-ds15.ds16,2393046648,1633529278326
23896,af,104.238.214.154,mitm-ra-ds8-ds13-ds15.ds8-ds15-dnskey15,2393046648,1633529278326


#### Remove Queries that Don't Belong to a Test

In [5]:
df_req_qsplit_clean = df_req_qsplit.dropna()
seen_tests = df_req_qsplit_clean['test'].unique()
print(f"Seen tests ({len(seen_tests)}):\n{sorted(seen_tests)}".replace(",", ",\n"))
# sorted()

Seen tests (51):
['broken',
 'ecdsap256sha256',
 'finish',
 'mitm-ra-ds8-ds13-ds15-ds16.ds16',
 'mitm-ra-ds8-ds13-ds15-ds16.ds8-ds15-dnskey15',
 'mitm-ra-ds8-ds13-ds15-ds16.ds8-ds16-dnskey16',
 'mitm-ra-ds8-ds13-ds15.ds13-ds16-dnskey13-dnskey16',
 'mitm-ra-ds8-ds13-ds15.ds16',
 'mitm-ra-ds8-ds13-ds15.ds8-ds15-dnskey15',
 'mitm-ra-ds8-ds13.ds13-ds16-dnskey13-dnskey16',
 'mitm-ra-ds8-ds13.ds8-ds15-dnskey15',
 'mitm-ra.ds10-dnskey10',
 'mitm-ra.ds13-dnskey13',
 'mitm-ra.ds14-dnskey14',
 'mitm-ra.ds15-dnskey15',
 'mitm-ra.ds16-dnskey16',
 'mitm-ra.ds5-dnskey5',
 'mitm-ra.ds8-dnskey8',
 'mitm-rs13-ra.ds13-ds16-dnskey13-dnskey16',
 'mitm-rs13-ra.ds16',
 'mitm-rs13-ra.ds8-ds13-dnskey8',
 'mitm-rs13-ra.ds8-ds15-dnskey8-dnskey15',
 'mitm-rs13-ra.ds8-ds16-dnskey16',
 'mitm-rs15-ra.ds13-ds16-dnskey13-dnskey16',
 'mitm-rs15-ra.ds16',
 'mitm-rs15-ra.ds8-dnskey8',
 'mitm-rs15-ra.ds8-ds13-dnskey13',
 'mitm-rs15-ra.ds8-ds15-dnskey15',
 'mitm-rs15-ra.ds8-ds16-dnskey16',
 'mitm-rs15-ra.ds8-ds16-dnskey8-

#### Aggregate over Clients the Tests that had Queries at the Web Server

In [6]:
df_req_clients = df_req_qsplit_clean[['token', 'test', 'ip_server']]
df_req_clients

# for name, group in request_df_clients:
#     print(f"{name}\n{group}\n")

,token,test,ip_server
0,730252720,mitm-rs8-ra.ds13-dnskey13,104.238.214.154
1,730252720,mitm-rs8-ra.ds13-ds16-dnskey13-dnskey16,104.238.214.154
2,730252720,mitm-ra-ds8-ds13-ds15.ds16,104.238.214.154
3,2977949033,mitm-ra-ds8-ds13.ds13-ds16-dnskey13-dnskey16,104.238.214.154
4,2977949033,mitm-ra-ds8-ds13.ds8-ds15-dnskey15,104.238.214.154
...,...,...,...
23893,2393046648,mitm-ra-ds8-ds13.ds13-ds16-dnskey13-dnskey16,104.238.214.154
23894,2393046648,mitm-ra-ds8-ds13.ds8-ds15-dnskey15,104.238.214.154
23895,2393046648,mitm-ra-ds8-ds13-ds15.ds16,104.238.214.154
23896,2393046648,mitm-ra-ds8-ds13-ds15.ds8-ds15-dnskey15,104.238.214.154


In [7]:
df_req_clients_queries_groupby = df_req_clients[['token', 'test', 'ip_server']].groupby('token')

# dirty but does the job...
newdf_v = []
for name, group in df_req_clients_queries_groupby:
    assert len(set(group['token'].values)) == 1
    token = group['token'].values[0]
    requests_present = dict((test_name, test_name in group['test'].values) for test_name in TEST_NAMES)
    newdf_v.append({'token': token, **requests_present})
df_request_presence = pd.DataFrame(newdf_v)
df_request_presence = df_request_presence.set_index('token')
df_request_presence

,mitm-rs15-ra.ds8-ds16-dnskey16,mitm-rs16-ra.ds13-ds16-dnskey16,mitm-rs8-ra.ds13-dnskey13,mitm-ra.ds5-dnskey5,mitm-rs15-ra.ds8-ds16-dnskey8-dnskey16,mitm-ra.ds14-dnskey14,mitm-ra-ds8-ds13.ds8-ds15-dnskey15,mitm-ra-ds8-ds13-ds15.ds13-ds16-dnskey13-dnskey16,mitm-rs8-ra.ds13-ds16-dnskey13-dnskey16,mitm-rs8-ra.ds8-dnskey8,...,mitm-rs8-ra.ds13-ds16-dnskey16,mitm-ra.ds10-dnskey10,mitm-rs16-ra.ds8-ds13-dnskey8,mitm-rs8-ra.ds16,mitm-rs16-ra.ds13-ds16-dnskey13-dnskey16,broken,mitm-rs16-ra.ds8-ds15-dnskey8-dnskey15,mitm-rs16-ra.ds13-ds15-dnskey15,mitm-rs8-ra.ds8-ds16-dnskey16,mitm-ra-ds8-ds13-ds15.ds16
token,,,,,,,,,,,,,,,,,,,,,
1005031220,True,False,False,False,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,True
1005416789,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
1007231486,False,False,False,False,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,True
1009635586,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1011222993,True,True,True,False,True,False,False,True,True,False,...,True,False,True,False,True,False,True,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991238089,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
992748482,False,False,False,False,False,False,True,False,False,False,...,False,False,False,True,False,False,False,False,False,False
996220222,True,True,True,False,True,False,False,True,True,False,...,True,False,True,True,True,False,True,True,False,True


### Infer Resolver Properties

#### Filter for Finished Sessions

In [8]:
df_finished_sessions = df_request_presence.copy(deep=True)
df_finished_sessions = df_finished_sessions[df_finished_sessions['session-finish'] == True]  # keep only those with finished session
# request_df_rp_fin = request_df_rp_fin.drop(columns=['finish'])
df_finished_sessions

,mitm-rs15-ra.ds8-ds16-dnskey16,mitm-rs16-ra.ds13-ds16-dnskey16,mitm-rs8-ra.ds13-dnskey13,mitm-ra.ds5-dnskey5,mitm-rs15-ra.ds8-ds16-dnskey8-dnskey16,mitm-ra.ds14-dnskey14,mitm-ra-ds8-ds13.ds8-ds15-dnskey15,mitm-ra-ds8-ds13-ds15.ds13-ds16-dnskey13-dnskey16,mitm-rs8-ra.ds13-ds16-dnskey13-dnskey16,mitm-rs8-ra.ds8-dnskey8,...,mitm-rs8-ra.ds13-ds16-dnskey16,mitm-ra.ds10-dnskey10,mitm-rs16-ra.ds8-ds13-dnskey8,mitm-rs8-ra.ds16,mitm-rs16-ra.ds13-ds16-dnskey13-dnskey16,broken,mitm-rs16-ra.ds8-ds15-dnskey8-dnskey15,mitm-rs16-ra.ds13-ds15-dnskey15,mitm-rs8-ra.ds8-ds16-dnskey16,mitm-ra-ds8-ds13-ds15.ds16
token,,,,,,,,,,,,,,,,,,,,,
1013762242,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1017200011,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
102556926,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,False,True,True,True,True
1029828803,True,True,True,False,True,False,False,True,True,False,...,True,False,True,True,True,False,True,True,False,True
1034585769,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990452317,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
990886273,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
991238089,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


#### Filter for DNSSEC Validation

See [RFC8624](https://datatracker.ietf.org/doc/html/rfc8624#section-3.1) for algo support specification.

In [10]:
df_validators = df_finished_sessions[~df_finished_sessions['broken']]
df_validators
# request_df_rp_fin_do = request_df_rp_fin

,mitm-rs15-ra.ds8-ds16-dnskey16,mitm-rs16-ra.ds13-ds16-dnskey16,mitm-rs8-ra.ds13-dnskey13,mitm-ra.ds5-dnskey5,mitm-rs15-ra.ds8-ds16-dnskey8-dnskey16,mitm-ra.ds14-dnskey14,mitm-ra-ds8-ds13.ds8-ds15-dnskey15,mitm-ra-ds8-ds13-ds15.ds13-ds16-dnskey13-dnskey16,mitm-rs8-ra.ds13-ds16-dnskey13-dnskey16,mitm-rs8-ra.ds8-dnskey8,...,mitm-rs8-ra.ds13-ds16-dnskey16,mitm-ra.ds10-dnskey10,mitm-rs16-ra.ds8-ds13-dnskey8,mitm-rs8-ra.ds16,mitm-rs16-ra.ds13-ds16-dnskey13-dnskey16,broken,mitm-rs16-ra.ds8-ds15-dnskey8-dnskey15,mitm-rs16-ra.ds13-ds15-dnskey15,mitm-rs8-ra.ds8-ds16-dnskey16,mitm-ra-ds8-ds13-ds15.ds16
token,,,,,,,,,,,,,,,,,,,,,
1017200011,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
102556926,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,False,True,True,True,True
1029828803,True,True,True,False,True,False,False,True,True,False,...,True,False,True,True,True,False,True,True,False,True
105095885,True,True,True,False,True,False,False,True,True,False,...,True,False,True,True,True,False,True,True,False,True
1053988585,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955208629,True,True,True,False,True,False,False,True,True,False,...,True,False,True,True,True,False,True,True,False,True
960736608,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
990452317,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Determine and Plot Statistics of Interest